# Use Spark for Python to load data and run SQL queries
This notebook introduces basic Spark concepts and helps you to start using Spark for Python.

Some familiarity with Python is recommended.

In this notebook, you'll use the publicly available **mtcars** data set from *Motor Trend* magazine to learn some basic Python. You'll learn how to load data, create a Spark DataFrame, aggregate data, run mathematical formulas, and run SQL queries against the data.


## Table of contents
This notebook contains these main sections:

1. [Load a DataFrame](#Load_a_DataFrame)
2. [Create an SQLContext](#Create_an_SQLContext)
3. [Create a Spark DataFrame](#Create_a_Spark_DataFrame)
4. [Aggregate data after grouping by columns](#Aggregate_data_after_grouping_by_columns)
5. [Operate on columns](#Operate_on_columns)
6. [Run SQL queries from the Spark DataFrame](#Run_SQL_queries_from_the_Spark_DataFrame)

<a id='Load_a_DataFrame'></a>
## 1. Load a DataFrame
A DataFrame is a distributed collection of data that is organized into named columns. The Python pandas DataFrame that you will create will load an external DataFrame called **mtcars**, which includes observations on the following 11 variables:

`[, 1]	mpg     Miles / (US) gallon`  
`[, 2]	cyl     Number of cylinders`  
`[, 3]	disp	Displacement (cu. in.)`  
`[, 4]	hp      Gross horsepower`  
`[, 5]	drat    Rear axle ratio`  
`[, 6]	wt      Weight (1000 lbs)`  
`[, 7]	qsec    1/4 mile time (seconds)`  
`[, 8]	vs      0 = V-engine, 1 = straight engine`  
`[, 9]	am      Transmission (0 = automatic, 1 = manual)`  
`[,10]	gear    Number of forward gears`  
`[,11]	carb    Number of carburetors`

In [1]:
import pandas as pd
mtcars = pd.read_csv('https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv')

Preview the first 3 rows of the DataFrame by using the `head()` method:

In [2]:
mtcars.head(3)

,car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1


<a id='Create_an_SQLContext'></a>
## 2. Create an SQLContext
To work with a DataFrame, you need an SQLContext class object, and to create a basic one, all you need is a SparkContext. A SparkContext represents the connection to a Spark cluster, and a SparkContext class object named sc, which has been created for you, is used to initialize the SQLContext:

In [3]:
sqlContext = SQLContext(sc)

<a id='Create_a_Spark_DataFrame'></a>
## 3. Create a Spark DataFrame
Using the SQLContext class object and the loaded local DataFrame, create a Spark DataFrame and print the schema, or structure, of the DataFrame:

In [4]:
sdf = sqlContext.createDataFrame(mtcars) 
sdf.printSchema()

root
 |-- car: string (nullable = true)
 |-- mpg: double (nullable = true)
 |-- cyl: long (nullable = true)
 |-- disp: double (nullable = true)
 |-- hp: long (nullable = true)
 |-- drat: double (nullable = true)
 |-- wt: double (nullable = true)
 |-- qsec: double (nullable = true)
 |-- vs: long (nullable = true)
 |-- am: long (nullable = true)
 |-- gear: long (nullable = true)
 |-- carb: long (nullable = true)



Display the content of the Spark DataFrame:

In [5]:
!pip list --isolated

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
appdirs (1.4.2)
Babel (2.1.1)
backports-abc (0.4)
backports.shutil-get-terminal-size (1.0.0)
backports.ssl-match-hostname (3.4.0.2)
basemap (1.0.7)
beautifulsoup4 (4.4.1)
biopython (1.66)
bitarray (0.8.1)
bokeh (0.10.0)
brunel (2.3)
cdsax-jupyter-extensions (0.1)
certifi (2015.11.20.1)
cognitive-assistant (1.0.30)
configparser (3.5.0)
cycler (0.9.0)
Cython (0.23.4)
debtcollector (1.1.0)
decorator (4.0.6)
descartes (1.0.1)
dill (0.2.5)
dlaas-client (0.1.29)
entrypoints (0.2.2)
enum (0.4.6)
enum34 (1.1.5)
extension-utils (0.1.590)
extras (0.0.3)
fixtures (2.0.0)
Flask (0.10.1)
funcsigs (0.4)
functools32 (3.2.3.post2)
future (0.15.2)
futures (3.0.5)
geopy (1.11.0)
ibmdbpy (0.1.4)
ip-associations-python-novaclient-ext (0.1)
ipykernel (4.3.1)
ipython (4.2.0)
ipython-genutils (0.1.0

In [ ]:
sdf.show(32)

Try different ways of retrieving subsets of the data. For example, get the first 5 values in the **mpg** column:

In [ ]:
sdf.select('mpg').show(5)

Filter the DataFrame to retain only rows with **mpg** values that are less than 18:

In [ ]:
sdf.filter(sdf['mpg'] < 18).show()

<a id='Aggregate_data_after_grouping_by_columns'></a>
## 4. Aggregate data after grouping by columns
Spark DataFrames support a number of common functions to aggregate data after grouping. For example, you can compute the average weight of cars as a function of the number of cylinders:

In [ ]:
sdf.groupby(['cyl'])\
.agg({"wt": "AVG"})\
.show()

You can also sort the output from the aggregation to determine the most popular cylinder configuration in the DataFrame:

In [ ]:
car_counts = sdf.groupby(['cyl'])\
.agg({"wt": "count"})\
.sort("count(wt)", ascending=False)\
.show()

<a id='Operate_on_columns'></a>
## 5. Operate on columns
Python provides a number of functions that you can apply directly to columns for data processing. In the following example, a basic arithmetic function converts lbs to metric tons:

In [ ]:
sdf.withColumn('wtTon', sdf['wt'] * 0.45).select('car','wt','wtTon').show(6)

<a id='Run_SQL_queries_from_the_Spark_DataFrame'></a>
## 6. Run SQL queries from the Spark DataFrame
You can register a Spark DataFrame as a temporary table and then run SQL queries over the data. The `sql` function enables an application to run SQL queries programmatically and returns the result as a DataFrame:

In [ ]:
sdf.registerTempTable("cars")

highgearcars = sqlContext.sql("SELECT car, gear FROM cars WHERE gear >= 5")
highgearcars.show()    

## That's it!
You successfully completed this notebook! You learned how to load a DataFrame, view and filter the data, aggregate the data, perform operations on the data in specific columns, and run SQL queries against the data. For more information about Spark, see the [Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html).

## Want to learn more?
### Free courses on <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu/" rel="noopener noreferrer" target="_blank">Big Data University</a>: <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu" rel="noopener noreferrer" target="_blank"><img src = "https://ibm.box.com/shared/static/xomeu7dacwufkoawbg3owc8wzuezltn6.png" width=600px> </a>

### Authors

**Saeed Aghabozorgi**, PhD, is a Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge. He is a researcher in the data mining field and an expert in developing advanced analytic methods like machine learning and statistical modelling on large data sets.

**Polong Lin** is a Data Scientist at IBM in Canada. Under the Emerging Technologies division, Polong is responsible for educating the next generation of data scientists through Big Data University. Polong is a regular speaker in conferences and meetups, and holds an M.Sc. in Cognitive Psychology.

Copyright © 2016 Big Data University. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/" rel="noopener noreferrer" target="_blank">MIT License</a>.